<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-13 12:43:48
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: 49.02 K (0.37%)
Current PnL: -21.89 L (-14.89%)
CY Booked + Current PnL: -13.62 L (-9.26%)
-------------------
Total profit:  2.61 L
Total loss:  -24.50 L
-------------------
Total Booked + Current PnL: 13.36 L (11.13%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.22%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.36 L (61.18%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.02,111944.0,7632.0,9023.0,4.89,7.32,8.06,15.96,11.0,0.85,0.86,25.45,X5K,ATH,METALS
68,SBIN,760.30,863.00,-12.30,M-LC,3.85,217936.0,16456.0,10766.0,0.22,8.17,4.94,13.51,61.0,1.53,1.67,21.79,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-15.30,H-LC,5.59,169334.0,13157.0,12937.0,6.36,8.42,7.64,16.71,19.0,1.02,1.30,27.17,X40N,BTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,10.86,H-MC,13.95,158352.0,32152.0,14030.0,-1.10,25.48,8.86,36.59,111.0,2.29,1.21,41.18,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-25.14,H-SC,9.63,223577.0,30557.0,17484.0,0.98,15.83,7.82,24.89,134.0,1.75,1.71,42.87,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DABUR,505.20,735.00,-10.32,H-MC,3.74,195273.0,-1755.0,91368.0,-0.44,-0.89,46.79,45.49,103.0,-0.02,1.50,12.03,XY24,BTT,FMCG
90,VOLTAS,1278.28,1918.49,-9.19,H-MC,7.67,190575.0,-1167.0,97193.0,2.05,-0.61,51.00,50.08,101.0,-0.01,1.46,6.15,XY25,ATH,AC
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.05,100652.0,148.0,30487.0,1.82,0.15,30.29,30.49,80.0,0.00,0.77,32.60,MH,ATH,METALS
92,WIPRO,243.46,420.00,-14.70,M-LC,7.16,149879.0,-1066.0,110521.0,0.01,-0.71,73.74,72.51,54.0,-0.01,1.15,5.27,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-19.44,H-LC,2.34,214610.0,10110.0,31312.0,2.95,4.94,14.59,20.25,10.0,0.32,1.64,14.11,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.06,H-LC,2.82,169531.0,-31855.0,119468.0,0.60,-15.82,70.47,43.51,27.0,-0.27,1.30,18.01,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-16.75,H-LC,3.60,128103.0,-137323.0,194230.0,0.35,-51.74,151.62,21.44,8.0,-0.71,0.98,3.37,XY25,NTT,FMCG
53,LTIM,5564.16,7230.2,-7.80,H-LC,3.78,183132.0,-17178.0,77154.0,-0.39,-8.58,42.13,29.94,17.0,-0.22,1.40,26.52,X200,ATH,IT
4,APOLLOHOSP,7098.95,8285.0,-15.30,H-LC,5.59,169334.0,13157.0,12937.0,6.36,8.42,7.64,16.71,19.0,1.02,1.30,27.17,X40N,BTT,HEALTHCARE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,7098.95,8285.00,-15.30,H-LC,5.59,169334.0,13157.0,12937.0,6.36,8.42,7.64,16.71,19.0,1.02,1.30,27.17,X40N,BTT,HEALTHCARE
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.86,72326.0,-41223.0,81273.0,5.37,-36.30,112.37,35.27,268.0,-0.51,0.55,77.02,XR,NTT,HOTELS
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.02,111944.0,7632.0,9023.0,4.89,7.32,8.06,15.96,11.0,0.85,0.86,25.45,X5K,ATH,METALS
21,CIPLA,1492.70,1795.00,-19.44,H-LC,2.34,214610.0,10110.0,31312.0,2.95,4.94,14.59,20.25,10.0,0.32,1.64,14.11,X40N,BTT,PHARMA
70,SHALBY,261.39,327.00,979.74,M-SC,7.07,146819.0,-34324.0,79796.0,2.85,-18.95,54.35,25.10,232.0,-0.43,1.13,17.66,XY24,NTT,HEALTHCARE


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,1550.24,2096.00,-38.58,M-SC,6.56,91781.0,-36889.0,82190.0,-2.64,-28.67,89.55,35.20,209.0,-0.45,0.70,0.00,X40,NTT,FOOTWEAR
59,QUESS,296.83,986.00,-40.50,M-SC,42.04,58834.0,-6172.0,157099.0,-2.45,-9.49,267.02,232.18,198.0,-0.04,0.45,0.54,XY24,NTT,MISC
29,EASEMYTRIP,18.12,26.40,-10.04,M-SC,24.18,86651.0,-90164.0,170962.0,-2.42,-50.99,197.30,45.70,197.0,-0.53,0.66,0.79,XY24,NTT,TRAVEL
30,GREENPANEL,375.16,537.00,195.87,M-SC,5.74,131980.0,-51098.0,130079.0,-1.90,-27.91,98.56,43.14,230.0,-0.39,1.01,24.15,XY24,NTT,MISC
24,COLPAL,2714.63,3767.14,-7.18,H-MC,11.71,168831.0,-42910.0,125002.0,-1.69,-20.27,74.04,38.77,86.0,-0.34,1.29,0.00,XY25,ATH,FMCG


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,93.60,M-SC,10.52,83368.0,-17409.0,17499.0,0.54,-17.27,20.99,0.09,243.0,-0.99,0.64,9.53,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,83.20,H-SC,20.56,116073.0,-26406.0,45930.0,-0.42,-18.53,39.57,13.70,163.0,-0.57,0.89,37.00,OX40N,NTT,BANKS
49,KANSAINER,299.63,340.0,-69.07,H-SC,2.51,213831.0,-55836.0,92161.0,-0.32,-20.71,43.10,13.47,142.0,-0.61,1.64,8.15,XY24,NTT,PAINTS
20,CERA,7989.07,9475.0,-20.53,H-SC,3.09,113454.0,-30349.0,57101.0,-0.83,-21.10,50.33,18.60,148.0,-0.53,0.87,25.20,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,108.71,M-SC,3.11,137020.0,-37539.0,37571.0,0.54,-21.50,27.42,0.02,221.0,-1.00,1.05,18.82,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,15.68,236761.0,-59173.0,153090.0,-0.18,-20.00,64.66,31.73,2.0,-0.39,1.81,1.08,X40,BTT,IT
45,INFY,1461.46,2275.00,-22.11,H-LC,10.63,255451.0,-6150.0,151763.0,0.21,-2.35,59.41,55.67,4.0,-0.04,1.96,3.61,X40,BTT,IT
89,VBL,492.64,671.64,-8.45,H-LC,10.42,277904.0,12864.0,83427.0,0.45,4.85,30.02,36.33,5.0,0.15,2.13,18.80,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.91,H-LC,10.18,197398.0,-19121.0,112023.0,-0.20,-8.83,56.75,42.91,7.0,-0.17,1.51,3.38,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-16.75,H-LC,3.60,128103.0,-137323.0,194230.0,0.35,-51.74,151.62,21.44,8.0,-0.71,0.98,3.37,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,10.86,H-MC,13.95,158352.0,32152.0,14030.0,-1.1,25.48,8.86,36.59,111.0,2.29,1.21,41.18,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,10.86,H-MC,13.95,158352.0,32152.0,14030.0,-1.10,25.48,8.86,36.59,111.0,2.29,1.21,41.18,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,9.79,H-SC,12.69,130662.0,5365.0,53637.0,0.71,4.28,41.05,47.09,141.0,0.10,1.00,63.53,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,2.13,H-MC,7.15,176420.0,11750.0,56225.0,1.07,7.14,31.87,41.28,91.0,0.21,1.35,15.69,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-4.94,H-SC,3.44,107957.0,5953.0,37968.0,2.30,5.84,35.17,43.06,123.0,0.16,0.83,23.95,X40N,ATH,MISC
35,HEROMOTOCO,4311.81,5949.07,-10.13,H-MC,7.40,166775.0,15862.0,41444.0,2.58,10.51,24.85,37.97,92.0,0.38,1.28,38.47,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-10.13,H-MC,7.40,166775.0,15862.0,41444.0,2.58,10.51,24.85,37.97,92.0,0.38,1.28,38.47,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.65,H-SC,12.19,134954.0,11618.0,122147.0,0.99,9.42,90.51,108.45,119.0,0.10,1.03,33.88,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,9.79,H-SC,12.69,130662.0,5365.0,53637.0,0.71,4.28,41.05,47.09,141.0,0.10,1.00,63.53,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-44.45,M-SC,4.00,93818.0,2320.0,76434.0,0.63,2.54,81.47,86.07,219.0,0.03,0.72,35.72,XR,NTT,DURABLES
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.05,100652.0,148.0,30487.0,1.82,0.15,30.29,30.49,80.0,0.00,0.77,32.60,MH,ATH,METALS


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.86,72326.0,-41223.0,81273.0,5.37,-36.30,112.37,35.27,268.0,-0.51,0.55,77.02,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-2.99,H-SC,16.26,90330.0,-10633.0,105162.0,1.32,-10.53,116.42,93.63,149.0,-0.10,0.69,34.54,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-18.70,H-SC,14.98,92355.0,-5625.0,27494.0,0.93,-5.74,29.77,22.32,152.0,-0.20,0.71,33.95,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-19.08,H-LC,6.02,111944.0,7632.0,9023.0,4.89,7.32,8.06,15.96,11.0,0.85,0.86,25.45,X5K,ATH,METALS
84,UJJIVANSFB,52.77,60.00,83.20,H-SC,20.56,116073.0,-26406.0,45930.0,-0.42,-18.53,39.57,13.70,163.0,-0.57,0.89,37.00,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SHALBY,261.39,327.00,979.74,M-SC,7.07,146819.0,-34324.0,79796.0,2.85,-18.95,54.35,25.10,232.0,-0.43,1.13,17.66,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-10.13,H-MC,7.40,166775.0,15862.0,41444.0,2.58,10.51,24.85,37.97,92.0,0.38,1.28,38.47,AR,ATH,AUTO
51,LALPATHLAB,2838.53,3545.00,-25.14,H-SC,9.63,223577.0,30557.0,17484.0,0.98,15.83,7.82,24.89,134.0,1.75,1.71,42.87,X40N,NTT,HEALTHCARE
89,VBL,492.64,671.64,-8.45,H-LC,10.42,277904.0,12864.0,83427.0,0.45,4.85,30.02,36.33,5.0,0.15,2.13,18.80,X40N,ATH,FMCG
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.86,72326.0,-41223.0,81273.0,5.37,-36.30,112.37,35.27,268.0,-0.51,0.55,77.02,XR,NTT,HOTELS


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.44
1,25,41.14
2,50,70.95


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.42
LC    34.28
MC    26.27
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.95
X40N     15.52
X40      13.31
XY25     10.66
XR        8.99
AR        8.41
OX40N     7.53
X5K       2.29
MH        1.77
X200      1.40
SR        1.14
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.91
H-SC    24.53
H-MC    23.31
M-SC    13.41
M-LC     7.27
M-MC     2.62
L-SC     1.48
L-LC     1.10
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.22,-11.09,46.80
IT,11.93,-19.75,79.55
FINANCE,8.32,-20.07,64.49
BANKS,7.64,-14.54,63.17
MISC,5.88,-19.84,88.54
PAINTS,5.66,-18.34,41.66
HEALTHCARE,5.35,5.95,17.80
ELECTRICAL,4.69,-14.76,64.46
AUTO,4.49,-15.33,65.51


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2862423.0,23
XR,1121598.0,13
AR,1110446.0,10
X40,833392.0,10
XY25,656863.0,8
X40N,579732.0,12
OX40N,475956.0,10
SR,194565.0,2
X5K,139887.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2816247.0,28
M-SC,1495023.0,17
H-LC,1354753.0,18
H-MC,1334793.0,18
M-LC,429759.0,5
M-MC,336461.0,2
L-SC,267027.0,3
L-MC,58651.0,1
L-LC,43426.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      939910.0      6
M-SC       XY24      871187.0      7
H-SC       AR        674509.0      5
           XR        531490.0      6
H-LC       X40       519861.0      5
H-MC       XY24      497283.0      5
M-MC       XY24      336461.0      2
H-LC       XY25      258541.0      3
           AR        233446.0      2
H-MC       X40       231341.0      4
           XY25      222195.0      2
H-SC       X40N      217249.0      4
H-LC       X40N      216754.0      5
H-SC       OX40N     204887.0      4
           SR        194565.0      2
M-LC       XY24      177608.0      2
L-SC       XR        177059.0      2
M-SC       AR        161047.0      2
H-MC       X40N      145729.0      3
M-SC       OX40N     132608.0      4
M-LC       X5K       130864.0      1
M-SC       XR        126056.0      2
           XY25      121935.0      1
H-MC       XR        117821.0      1
M-LC       XR        110521.0      1
L-SC       OX40N      89968.0      1
M-SC       X40        82190.0      1
H-LC       X200       77154.0      1
L-MC       XR         58651.0      1
H-SC       MH         53637.0      1
H-MC       OX40N      48493.0      1
L-LC       XY25       43426.0      1
H-MC       AR         41444.0      1
H-LC       XY24       39974.0      1
H-MC       MH         30487.0      1
M-LC       XY25       10766.0      1
H-LC       X5K         9023.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
